<a href="https://colab.research.google.com/github/rdkdaniel/Game-AI/blob/main/Solving_and_Optimizing_Sokoban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**

In this notebook, I research and implement models/algorithms to first solve Sokobna.

Thereafter, I try to optimize these solutions as well as existing ones.

Key links: 
*   https://github.com/joriswit/YASS
*   https://timallanwheeler.com/blog/2022/01/19/basic-search-algorithms-on-sokoban/

